In [ ]:
# Custom trained model for Number plate detection based on YOLOv8. data set = https://www.kaggle.com/datasets/saisirishan/indian-vehicle-dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
import os
import glob

base_data_path = "/content/drive/MyDrive/State-wise_OLX"
image_paths = glob.glob(f"{base_data_path}/**/*.jpg", recursive=True)
xml_paths = [img.replace('.jpg','.xml') for img in image_paths]

image_xml_pais = [(img, xml) for img, xml in zip(image_paths, xml_paths) if os.path.exists(xml)]
print(f"Total valid image-xml pairs: {len(image_xml_pais)}")

Total valid image-xml pairs: 602


In [4]:
import xml.etree.ElementTree as ET

def convert_xml_to_yolo(xml_file, img_w, img_h):
  tree = ET.parse(xml_file)
  root = tree.getroot()

  yolo_lines = []
  for obj in root.findall("object"):
    bndbox = obj.find("bndbox")
    xmin = int(bndbox.find("xmin").text)
    ymin = int(bndbox.find("ymin").text)
    xmax = int(bndbox.find("xmax").text)
    ymax = int(bndbox.find("ymax").text)

    x_center = ((xmin + xmax) / 2) / img_w
    y_center = ((ymin + ymax) / 2) / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h

    yolo_lines.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
  return yolo_lines

In [5]:
from sklearn.model_selection import train_test_split
from PIL import Image
import shutil

train_set, val_set = train_test_split(image_xml_pais, test_size=0.2, random_state=42)

base_output = "/content/drive/MyDrive/state-wise-OLX-output"
folders = [
    f"{base_output}/images/train", f"{base_output}/images/val",
    f"{base_output}/labels/train", f"{base_output}/labels/val"
]
for folder in folders:
  os.makedirs(folder, exist_ok = True)

def process_and_save(pairs, img_dst, lbl_dst):
  for img_path, xml_path in pairs:
    try:
      im = Image.open(img_path)
      w,h = im.size

      lines = convert_xml_to_yolo(xml_path, w, h)
      base = os.path.basename(img_path).replace(".jpg","")
      shutil.copy(img_path, f"{img_dst}/{base}.jpg")
      with open(f"{lbl_dst}/{base}.txt","w") as f:
        f.write("\n".join(lines))
    except Exception as e:
      print(f"Skipping {img_path} due to error: {e}")

process_and_save(train_set, f"{base_output}/images/train", f"{base_output}/labels/train")
process_and_save(val_set, f"{base_output}/images/val", f"{base_output}/labels/val")

In [6]:
yaml_content = f"""
path: {base_output}
train: images/train
val: images/val
names:
  0: license_plate
"""

with open("number_plate.yaml", "w") as f:
    f.write(yaml_content)

In [7]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data = 'number_plate.yaml',
    epochs = 50,
    imgsz = 640,
    batch = 16,
    save = True,
    save_period = -1,
    val = True,
    name = 'number_plate_run'
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 84.7MB/s]


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=number_plate.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=number_plate_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

100%|██████████| 755k/755k [00:00<00:00, 17.1MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        120        120      0.999      0.992      0.995      0.863



50 epochs completed in 0.146 hours.
Optimizer stripped from runs/detect/number_plate_run/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/number_plate_run/weights/best.pt, 6.2MB

Validating runs/detect/number_plate_run/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.87it/s]


                   all        120        120          1      0.999      0.995      0.867
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 6.2ms postprocess per image
Results saved to runs/detect/number_plate_run


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x799edb8b2810>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [8]:
!cp /content/runs/detect/number_plate_run/weights/best.pt /content/drive/MyDrive/best_number_plate_model.pt